# Ensemble Techniques

## Environment Setup

### Imports

In [1]:
import os
os.sys.path.append("utils")

# Imports
import torch
import numpy as np
import pandas as pd
import nibabel as nib
from functools import reduce
import torch.nn.functional as F
from monai.metrics import DiceMetric
from monai.transforms import AsDiscrete
from skimage.measure import regionprops, label

### Config

In [2]:
seed = 33
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

channels = ['TC', 'WT', 'ET']

In [3]:
# Ensemble Inference
def ensemble_inference(dataframe, ensemble_function, threshold = 0.5, include_label = False, model = None, store_npz = False, model_name = None):

    # Transforms
    trans = AsDiscrete(threshold=threshold)

    # Dice Params
    dice_values, dice_values_tc, dice_values_wt, dice_values_et = [], [], [], []
    dice_metric = DiceMetric(include_background=True, reduction="mean")
    dice_metric_batch = DiceMetric(include_background=True, reduction="mean_batch")

    # Biometrics Params
    ids = []
    pred_paths = []
    gt_nm, pred_nm = {'TC': [], 'WT': [], 'ET': []}, {'TC': [], 'WT': [], 'ET': []}
    gt_v, pred_v = {'TC': [], 'WT': [], 'ET': []}, {'TC': [], 'WT': [], 'ET': []}

    # Iterate over the dataframe
    for i in range(len(dataframe)):

        # Subject & Label
        subject_id = dataframe['SubjectID'][i]
        load_label = dataframe['GT'][i]
        
        # Images
        load_ahnet = dataframe['AHNet'][i]
        load_segresnet = dataframe['SegResNet'][i]
        load_unet = dataframe['UNet'][i]
        load_unetr = dataframe['UNETR'][i]
        
        # Params 
        image_voxel_volume = np.prod((1,1,1))
        label_voxel_volume = np.prod((1,1,1))

        # Load Images and Labels
        ahnet_image = [np.load(x)['arr_0'] for x in load_ahnet]
        segresnet_image = [np.load(x)['arr_0'] for x in load_segresnet]
        unetr_image = [np.load(x)['arr_0'] for x in load_unetr]
        img_label = [np.load(x)['arr_0'] for x in load_label] 

        # To Tensor
        ahnet_image = [torch.from_numpy(x) for x in ahnet_image]
        segresnet_image = [torch.from_numpy(x) for x in segresnet_image]
        unetr_image = [torch.from_numpy(x) for x in unetr_image]
        img_label = [torch.from_numpy(x) for x in img_label]
            
        # Stack Images and Label
        ahnet_image = torch.stack(ahnet_image, dim = 0).unsqueeze(0)
        segresnet_image = torch.stack(segresnet_image, dim = 0).unsqueeze(0)
        unetr_image = torch.stack(unetr_image, dim = 0).unsqueeze(0)
        img_label = torch.stack(img_label, dim = 0).unsqueeze(0)

        # Ensemble Function
        img = None
        if include_label:
            img = ensemble_function([ahnet_image, segresnet_image, unetr_image], img_label)
        elif model is not None:
            img = ensemble_function([ahnet_image, segresnet_image, unetr_image], model)
        else:
            img = ensemble_function([ahnet_image, segresnet_image, unetr_image])

        # Save NPZ
        if store_npz:
            np.savez_compressed(f'./outputs/Ensemble/pred_{model_name}/pred_{subject_id}_TC.npz', img[0][0])
            np.savez_compressed(f'./outputs/Ensemble/pred_{model_name}/pred_{subject_id}_WT.npz', img[0][1])
            np.savez_compressed(f'./outputs/Ensemble/pred_{model_name}/pred_{subject_id}_ET.npz', img[0][2])
            pred_paths.append([f'./outputs/Ensemble/pred_{model_name}/pred_{subject_id}_TC.npz', 
                               f'./outputs/Ensemble/pred_{model_name}/pred_{subject_id}_WT.npz',
                               f'./outputs/Ensemble/pred_{model_name}/pred_{subject_id}_ET.npz'])

        # Discretizise
        img = trans(img)
        img_label = trans(img_label)

        # Dice Metric
        dice_metric(y_pred=img, y=img_label)
        dice_score = dice_metric.aggregate()
        dice_values.append(dice_score.item())
        dice_metric.reset()
            
		# Batch Dice
        dice_metric_batch(y_pred=img, y=img_label)
        dice_batch = dice_metric_batch.aggregate()
        dice_values_tc.append(dice_batch[0].item())
        dice_values_wt.append(dice_batch[1].item())
        dice_values_et.append(dice_batch[2].item())
        dice_metric_batch.reset()     

        # Biometrics
        for j, channel in enumerate(channels):
            # Image
            props = regionprops(label(nib.Nifti1Image(img[0][j].cpu().numpy(), np.eye(4)).get_fdata()))
            volumes = [prop.area * image_voxel_volume for prop in props]
            pred_nm[channel].append(int(len(volumes)))
            pred_v[channel].append(int(np.sum(volumes)))
            # Label
            props = regionprops(label(nib.Nifti1Image(img_label[0][j].cpu().numpy(), np.eye(4)).get_fdata()))
            volumes = [prop.area * label_voxel_volume for prop in props]
            gt_nm[channel].append(int(len(volumes)))
            gt_v[channel].append(int(np.sum(volumes)))

        # Subject ID
        ids.append(subject_id)
                
    # Excel
    df = pd.DataFrame({
        'SubjectID': ids,
		'Dice': dice_values,
		'Dice TC': dice_values_tc,
		'Dice WT': dice_values_wt,
		'Dice ET': dice_values_et,
		'Pred NM TC': pred_nm['TC'],
		'Pred NM WT': pred_nm['WT'],
		'Pred NM ET': pred_nm['ET'],
		'GT NM TC': gt_nm['TC'],
		'GT NM WT': gt_nm['WT'],
		'GT NM ET': gt_nm['ET'],
		'Pred V TC': pred_v['TC'],
		'Pred V WT': pred_v['WT'],
		'Pred V ET': pred_v['ET'],
		'GT V TC': gt_v['TC'],
		'GT V WT': gt_v['WT'],
		'GT V ET': gt_v['ET'],
	})
    
    if store_npz:
        df['Pred Paths'] = pred_paths
    
    return df

## Load Test Data

In [4]:
subject_ids = pd.read_csv('./data/TEST.csv')['SubjectID'].values

ah_segs, unet_segs, segresnet_segs, untr_segs, gt_segs = [], [], [], [], []
for sid in subject_ids:
    ah_channels, unet_channels, segresnet_channels, untr_channels, gt_channels = [], [], [], [], []
    for channel in ['TC', 'WT', 'ET']:
        ah_channels.append(f'./outputs/AHNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        unet_channels.append(f'./outputs/UNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        segresnet_channels.append(f'./outputs/SegResNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        untr_channels.append(f'./outputs/UNETR/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        gt_channels.append(f'./outputs/gt_segs/test_gt_segs/gt_{sid}_{channel}.npz')
    
    ah_segs.append(ah_channels)
    unet_segs.append(unet_channels)
    segresnet_segs.append(segresnet_channels)
    untr_segs.append(untr_channels)
    gt_segs.append(gt_channels)

# Dataframe
test_df = pd.DataFrame()
test_df['SubjectID'] = subject_ids
test_df['AHNet'] = ah_segs
test_df['UNet'] = unet_segs
test_df['SegResNet'] = segresnet_segs
test_df['UNETR'] = untr_segs
test_df['GT'] = gt_segs

test_df.head()

,SubjectID,AHNet,UNet,SegResNet,UNETR,GT
0,100214B,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100214B_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100214B_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100214B_ET.npz]"
1,100340A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100340A_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100340A_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100340A_ET.npz]"
2,100391A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100391A_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100391A_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100391A_ET.npz]"
3,100190B,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100190B_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100190B_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100190B_ET.npz]"
4,100142A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_1

## Ensemble 1 - Average

In [5]:
def avg(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.mean(stacked_predictions, dim=0)

#---------------------------------------------------------#

df = ensemble_inference(test_df, avg, threshold=0.4)
df.to_csv('./outputs/EnsembleNU/ensemble_avg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.642900,0.675335,0.684806,0.547999,7.870968,7.032258,16.096774,8.709677,8.709677,5.129032,16631.483871,16666.580645,13891.870968,17105.387097,17985.870968,13810.806452
std,0.252928,0.241508,0.246799,0.303004,7.830462,6.374344,21.357832,9.103822,9.103822,4.951376,25129.788233,25262.459670,22386.062110,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.523253,0.570072,0.575907,0.320578,3.000000,3.000000,3.000000,2.000000,2.000000,1.500000,982.500000,960.000000,551.500000,802.000000,802.000000,410.000000
50%,0.637359,0.703949,0.753131,0.577692,5.000000,5.000000,8.000000,6.000000,6.000000,4.000000,4718.000000,4724.000000,2852.000000,3847.000000,4138.000000,1944.000000
75%,0.855782,0.873027,0.898497,0.793281,9.500000,9.500000,21.500000,13.500000,13.500000,7.000000,30380.000000,30354.000000,28023.000000,26305.000000,27036.500000,23520.500000
max,0.949085,0.955557,0.958394,0.935284,34.000000,26.000000,99.000000,43.000000,43.000000,21.000000,95617.000000,95689.000000,84916.000000,102770.000000,105729.000000,92221.000000


## Ensemble 2 - Weighted Average (Mean Dice Weights)

In [6]:
def weigthed_avg(predictions):

    # Weights form validation set AHNet, SegResNet, UNet, UNETR
    weights = np.array([0.546402, 0.648790, 0.547957])
    weights = weights / np.sum(weights)
    weights = torch.tensor(weights).float()
    # Weighted average
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.sum(stacked_predictions * weights[:, None, None, None, None, None], dim=0)
     
#---------------------------------------------------------#

df = ensemble_inference(test_df, weigthed_avg, threshold=0.4)
df.to_csv('./outputs/EnsembleNU/ensemble_weighted_avg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.645398,0.676805,0.687453,0.552012,7.193548,6.967742,14.000000,8.709677,8.709677,5.129032,16555.032258,16586.225806,13788.322581,17105.387097,17985.870968,13810.806452
std,0.251027,0.240210,0.243862,0.301197,6.935029,6.635681,17.189144,9.103822,9.103822,4.951376,24977.822062,25104.340697,22205.619714,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.534398,0.576200,0.580152,0.331640,3.500000,3.000000,3.000000,2.000000,2.000000,1.500000,995.500000,970.000000,566.000000,802.000000,802.000000,410.000000
50%,0.631351,0.713736,0.762651,0.576646,5.000000,5.000000,7.000000,6.000000,6.000000,4.000000,4724.000000,4729.000000,2863.000000,3847.000000,4138.000000,1944.000000
75%,0.856444,0.873076,0.898498,0.794745,8.500000,8.500000,19.000000,13.500000,13.500000,7.000000,29923.500000,29944.500000,27641.000000,26305.000000,27036.500000,23520.500000
max,0.949815,0.956011,0.959204,0.935853,30.000000,27.000000,71.000000,43.000000,43.000000,21.000000,95005.000000,95181.000000,84273.000000,102770.000000,105729.000000,92221.000000


## Ensemble 3 - Median 

In [7]:
def median(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.median(stacked_predictions, dim=0).values

#---------------------------------------------------------#

df = ensemble_inference(test_df, median, threshold=0.4)
df.to_csv('./outputs/EnsembleNU/ensemble_median.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.637875,0.672492,0.680239,0.541232,8.709677,7.967742,9.612903,8.709677,8.709677,5.129032,16071.645161,16227.548387,13444.032258,17105.387097,17985.870968,13810.806452
std,0.254386,0.241583,0.246623,0.308399,7.751101,7.467636,11.238853,9.103822,9.103822,4.951376,24189.381509,24527.321182,21631.560320,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.529374,0.570772,0.579417,0.291528,3.500000,3.500000,2.500000,2.000000,2.000000,1.500000,974.000000,953.000000,556.000000,802.000000,802.000000,410.000000
50%,0.613806,0.693872,0.745872,0.567229,5.000000,5.000000,6.000000,6.000000,6.000000,4.000000,4626.000000,4656.000000,2851.000000,3847.000000,4138.000000,1944.000000
75%,0.853084,0.873338,0.894805,0.791979,12.500000,9.000000,12.000000,13.500000,13.500000,7.000000,28907.500000,29085.500000,26872.000000,26305.000000,27036.500000,23520.500000
max,0.949471,0.956106,0.957815,0.936082,30.000000,28.000000,47.000000,43.000000,43.000000,21.000000,92858.000000,93420.000000,82590.000000,102770.000000,105729.000000,92221.000000


## Ensemble 4 - Maximum Confidence 

In [8]:
def max_confidence(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.max(stacked_predictions, dim=0).values

#---------------------------------------------------------#

df = ensemble_inference(test_df, max_confidence, threshold=0.7)
df.to_csv('./outputs/EnsembleNU/ensemble_max_confidence.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.627460,0.664593,0.670343,0.537957,14.225806,12.354839,155.516129,8.709677,8.709677,5.129032,17815.870968,17703.354839,15239.096774,17105.387097,17985.870968,13810.806452
std,0.253009,0.237301,0.254247,0.294088,9.965640,9.112440,253.996177,9.103822,9.103822,4.951376,25973.511098,25893.051202,23651.232029,25565.209783,26912.616906,22937.645993
min,0.037298,0.082192,0.029703,0.000000,1.000000,2.000000,0.000000,1.000000,1.000000,0.000000,45.000000,28.000000,0.000000,44.000000,44.000000,0.000000
25%,0.443800,0.528188,0.501666,0.300136,8.500000,7.000000,14.500000,2.000000,2.000000,1.500000,1056.000000,1017.500000,593.500000,802.000000,802.000000,410.000000
50%,0.668926,0.747748,0.738101,0.564730,11.000000,9.000000,49.000000,6.000000,6.000000,4.000000,4778.000000,4772.000000,3160.000000,3847.000000,4138.000000,1944.000000
75%,0.844789,0.860551,0.896599,0.772265,15.000000,16.000000,145.500000,13.500000,13.500000,7.000000,31523.000000,31307.000000,29318.000000,26305.000000,27036.500000,23520.500000
max,0.943834,0.953132,0.955554,0.922817,41.000000,37.000000,1118.000000,43.000000,43.000000,21.000000,98422.000000,97834.000000,89299.000000,102770.000000,105729.000000,92221.000000


## Ensemble 5 - Majority Voting

In [9]:
def majority_voting(predictions):
    transforms =[AsDiscrete(threshold=0.3), AsDiscrete(threshold=0.5), AsDiscrete(threshold=0.2), AsDiscrete(threshold=0.4)]
    predictions = [transform(prediction) for transform, prediction in zip(transforms, predictions)]
    stacked_predictions = torch.stack(predictions, dim=0)  
    return torch.mode(stacked_predictions, dim=0).values

#---------------------------------------------------------#

majority_vote_df = ensemble_inference(test_df, majority_voting)
majority_vote_df.to_csv('./outputs/EnsembleNU/ensemble_majority_vote.csv', index=False)
majority_vote_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.641846,0.673063,0.684523,0.548373,8.774194,7.967742,9.677419,8.709677,8.709677,5.129032,16753.774194,16843.483871,14103.161290,17105.387097,17985.870968,13810.806452
std,0.246541,0.235789,0.237658,0.297688,8.321497,7.190196,10.734328,9.103822,9.103822,4.951376,25181.015711,25374.714886,22529.979089,25565.209783,26912.616906,22937.645993
min,0.018875,0.022727,0.033898,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,2.000000,3.000000,0.000000,44.000000,44.000000,0.000000
25%,0.530874,0.575143,0.587454,0.342862,3.000000,3.000000,3.000000,2.000000,2.000000,1.500000,1012.500000,984.500000,580.000000,802.000000,802.000000,410.000000
50%,0.639261,0.718425,0.767663,0.560427,5.000000,5.000000,6.000000,6.000000,6.000000,4.000000,4725.000000,4746.000000,3012.000000,3847.000000,4138.000000,1944.000000
75%,0.854089,0.873574,0.891474,0.792250,11.000000,10.000000,11.500000,13.500000,13.500000,7.000000,30727.000000,30834.000000,28597.000000,26305.000000,27036.500000,23520.500000
max,0.941908,0.948914,0.950959,0.928411,33.000000,30.000000,45.000000,43.000000,43.000000,21.000000,96470.000000,96708.000000,86145.000000,102770.000000,105729.000000,92221.000000


## Ensemble 6 - Dempster-Shafer Theory

In [10]:
def dempster_rule(mass1, mass2):

    K = 1 / (1 - mass1[..., 0] * mass2[..., 1] - mass1[..., 1] * mass2[..., 0])

    combined_mass = torch.zeros_like(mass1)
    combined_mass[..., 0] = K * (mass1[..., 0] * mass2[..., 0] + mass1[..., 0] * mass2[..., 2] + mass1[..., 2] * mass2[..., 0])
    combined_mass[..., 1] = K * (mass1[..., 1] * mass2[..., 1] + mass1[..., 1] * mass2[..., 2] + mass1[..., 2] * mass2[..., 1])
    combined_mass[..., 2] = K * (mass1[..., 0] * mass2[..., 1] + mass1[..., 1] * mass2[..., 0] + mass1[..., 2] * mass2[..., 2])
    
    return combined_mass

def dempster_shafer_inference(predictions):

    # Params
    predictions = [pred.squeeze(0) for pred in predictions]

    # Mass Beliefs for {tumor, non-tumor} - (3, 3, x, y, z) - (masses, channels, x, y, z)
    masses = []	
    for pred in predictions:
        epsilon = 1 - torch.max(pred) 
        tumor_mass = pred
        non_tumor_mass = 1 - pred
        uncertainty_mass = torch.full_like(pred, epsilon)
        masses.append(torch.stack([tumor_mass, non_tumor_mass, uncertainty_mass], dim=-1))

    combined_mass = reduce(dempster_rule, masses)

    # Normalized Mass
    combined_mass_sum = combined_mass.sum(dim=-1, keepdim=True)
    normalized_mass = combined_mass / combined_mass_sum
    
    return normalized_mass[..., 0].unsqueeze(0)   

#---------------------------------------------------------#

ds_df = ensemble_inference(test_df, dempster_shafer_inference, threshold=0.1)
ds_df.to_csv('./outputs/EnsembleNU/ensemble_dempster_shafer.csv', index=False)
ds_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.601745,0.633941,0.644781,0.505325,8.516129,8.419355,9.290323,8.709677,8.709677,5.129032,16802.903226,16908.709677,14467.838710,17105.387097,17985.870968,13810.806452
std,0.264539,0.254862,0.260568,0.308289,7.719546,7.873052,10.687044,9.103822,9.103822,4.951376,25970.063724,26162.844280,23447.012299,25565.209783,26912.616906,22937.645993
min,0.011385,0.022727,0.011429,0.000000,2.000000,1.000000,0.000000,1.000000,1.000000,0.000000,2.000000,1.000000,0.000000,44.000000,44.000000,0.000000
25%,0.456363,0.526884,0.549712,0.230807,3.500000,4.000000,2.500000,2.000000,2.000000,1.500000,850.000000,850.000000,490.000000,802.000000,802.000000,410.000000
50%,0.585188,0.640000,0.629866,0.503819,6.000000,6.000000,6.000000,6.000000,6.000000,4.000000,4183.000000,4317.000000,3003.000000,3847.000000,4138.000000,1944.000000
75%,0.849310,0.874172,0.884706,0.785920,11.000000,10.500000,10.000000,13.500000,13.500000,7.000000,31359.500000,31485.000000,29419.000000,26305.000000,27036.500000,23520.500000
max,0.937775,0.944772,0.947060,0.921493,31.000000,32.000000,47.000000,43.000000,43.000000,21.000000,98062.000000,98116.000000,88349.000000,102770.000000,105729.000000,92221.000000


## Ensemble 7 - CRF - No Energy Minimization

In [11]:
def crf_ne(predictions, sigma_spatial=3.0, sigma_bilateral=1.0, iterations=10, alpha=0.7):
    # Weighted Average
    weights = torch.tensor([0.546402, 0.648790, 0.547957]).float()
    weights = weights / weights.sum()
    stacked_preds = torch.stack(predictions, dim=0)
    avg_pred = torch.sum(stacked_preds * weights[:, None, None, None, None, None], dim=0)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    avg_pred = avg_pred.to(device)
    Q = avg_pred.clone()

    for _ in range(iterations):
        # Spatial
        Q_spatial = F.avg_pool3d(Q, kernel_size=3, stride=1, padding=1)

        # Bilateral
        diff = (Q.unsqueeze(2) - Q.unsqueeze(1)).pow(2)
        bilateral_weight = torch.exp(-diff / (2 * sigma_bilateral**2))
        Q_bilateral = (Q.unsqueeze(2) * bilateral_weight).sum(1) / bilateral_weight.sum(1)

        # Combine spatial and bilateral
        Q_new = Q * torch.exp((Q_spatial - Q) / sigma_spatial + (Q_bilateral - Q) / sigma_bilateral)
        Q_new = Q_new / Q_new.sum(dim=1, keepdim=True)

        Q = Q_new
        
    final_pred = alpha * avg_pred + (1 - alpha) * Q

    return final_pred.cpu()

#--------------------------------------------------------------------------------

df = ensemble_inference(test_df, crf_ne, threshold=0.4, store_npz=True, model_name='crf_ne')
df.to_csv('./outputs/EnsembleNU/ensemble_crf_ne.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.643267,0.676059,0.684608,0.548649,7.193548,6.645161,11.483871,8.709677,8.709677,5.129032,16231.225806,16279.806452,13430.064516,17105.387097,17985.870968,13810.806452
std,0.253890,0.242339,0.247343,0.304838,6.920594,6.457803,13.220870,9.103822,9.103822,4.951376,24534.594601,24696.136302,21763.150106,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.533568,0.578342,0.581405,0.323960,3.500000,3.000000,3.000000,2.000000,2.000000,1.500000,965.000000,935.000000,529.500000,802.000000,802.000000,410.000000
50%,0.635036,0.703195,0.752616,0.582860,5.000000,4.000000,6.000000,6.000000,6.000000,4.000000,4661.000000,4643.000000,2750.000000,3847.000000,4138.000000,1944.000000
75%,0.855568,0.871944,0.898139,0.796715,8.500000,8.500000,16.500000,13.500000,13.500000,7.000000,28962.000000,28988.000000,26655.000000,26305.000000,27036.500000,23520.500000
max,0.951052,0.956996,0.959822,0.937410,30.000000,26.000000,51.000000,43.000000,43.000000,21.000000,93512.000000,93718.000000,82698.000000,102770.000000,105729.000000,92221.000000


## Ensemble 8 - CRF - Energy Minimization

In [13]:
def crf(predictions, iterations=5, sigma_spatial=3.0, sigma_bilateral=1.0, learning_rate=1e-3, alpha=0.7):
    # Weighted Average
    weights = torch.tensor([0.546402, 0.648790, 0.547957]).float()
    weights = weights / weights.sum()
    stacked_preds = torch.stack(predictions, dim=0)
    avg_pred = torch.sum(stacked_preds * weights[:, None, None, None, None, None], dim=0)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    avg_pred = avg_pred.to(device)

    Q = avg_pred.clone().requires_grad_(True)

    for _ in range(iterations):
        # Compute energy
        unary_energy = -torch.sum(avg_pred * torch.log(Q + 1e-10))
        
        # Spatial pairwise energy
        Q_spatial = F.avg_pool3d(Q, kernel_size=3, stride=1, padding=1)
        spatial_energy = torch.sum((Q - Q_spatial).pow(2)) / (2 * sigma_spatial**2)
        
        # Bilateral pairwise energy
        diff = (Q.unsqueeze(2) - Q.unsqueeze(1)).pow(2)
        bilateral_weight = torch.exp(-diff / (2 * sigma_bilateral**2))
        Q_bilateral = (Q.unsqueeze(2) * bilateral_weight).sum(1) / bilateral_weight.sum(1)
        bilateral_energy = torch.sum((Q - Q_bilateral).pow(2)) / (2 * sigma_bilateral**2)
        
        # Total energy
        energy = unary_energy + spatial_energy + bilateral_energy
        
        # Update Q
        energy.backward()
        with torch.no_grad():
            Q -= learning_rate * Q.grad
            Q.grad.zero_()
        Q = F.softmax(Q, dim=1)
        Q = Q.detach().requires_grad_(True)

    final_pred = alpha * avg_pred + (1 - alpha) * Q
    
    return final_pred.detach().cpu()

#--------------------------------------------------------------------------------

df = ensemble_inference(test_df, crf, threshold=0.4, store_npz=True, model_name='crf')
df.to_csv('./outputs/EnsembleNU/ensemble_crf.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.642601,0.675014,0.683434,0.548793,7.580645,6.903226,11.806452,8.709677,8.709677,5.129032,16208.903226,16254.000000,13433.935484,17105.387097,17985.870968,13810.806452
std,0.254197,0.242991,0.247759,0.304729,7.297370,6.410173,13.731762,9.103822,9.103822,4.951376,24532.480847,24693.675019,21767.654772,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.533431,0.577694,0.581337,0.323906,2.500000,3.000000,3.000000,2.000000,2.000000,1.500000,961.500000,934.500000,530.500000,802.000000,802.000000,410.000000
50%,0.637398,0.703398,0.752826,0.582716,5.000000,5.000000,6.000000,6.000000,6.000000,4.000000,4655.000000,4638.000000,2751.000000,3847.000000,4138.000000,1944.000000
75%,0.855494,0.872075,0.898147,0.796974,9.000000,9.000000,17.000000,13.500000,13.500000,7.000000,28955.500000,28980.500000,26658.000000,26305.000000,27036.500000,23520.500000
max,0.951048,0.956983,0.959822,0.937292,30.000000,25.000000,53.000000,43.000000,43.000000,21.000000,93493.000000,93684.000000,82721.000000,102770.000000,105729.000000,92221.000000
